In [9]:

macro_names = 'tests/main.tex'
tex_file = 'tests/theory.tex'

In [74]:
process_file('tests/theory.tex',macs)
process_file('tests/methods.tex',macs)

In [11]:
macro_list = []
with open(macro_names,'r') as f:
    for line in f.readlines():
        if 'newcommand' in line:
            macro_list.append(line)
            #macro_list.append(Macro(line))

In [146]:
class TexFile():
    def __init__(self, filname):
        self.filename = filename
        with open(filename,'r') as f:
            self.content = f.read()
        
        input_files = self.content.split('\input')
        if len(input_files > 1):
            self.extra_files = [TexFile(grab_argument(line,1)[0] + '.tex') for line in input_files]
        self.macro_list = [Macro(line) for line in self.content.split('\\newcommand')[1:]]

In [129]:
class Macro():
    def __init__(self,input_str):
        self.original = input_str.strip()
        temp = input_str.strip().split('newcommand')
        self.re = False
        if 're' in temp[0]:
            self.re = True            
        self.name, temp = self.parse_brackets(temp[1].strip())
        self.numargs, temp = self.grab_num_args(temp)
        self.definition = self.grab_result(temp)
        
    def parse_brackets(self, str_in):
        res = '{'.join(str_in.split('{')[1:]).split('}')
        name = res[0]
        res = '}'.join(res[1:])
        return name,res
    def grab_num_args(self, str_in):
        if str_in[0] != '[':
            return 0,str_in
        else: 
            return int(str_in[1]),']'.join(str_in.split(']')[1:])

    def grab_result(self, str_in,num_args=0):
        res = '}'.join('{'.join(str_in.split('{')[1:]).split('}')[:-1])
        return res
    def replace(self, args):
        if self.numargs == 0:
            return self.definition
        mac = self.definition
        for i in range(self.numargs):
            mac = mac.replace('#'+str(i+1),args[i])
        return mac
    def parse_arguments(self, input_str):
    
        temp = input_str.split(self.name)
        left_str = temp[0]
        right_str = self.name.join(temp[1:])
        num = 0
        istart = 0
        args = []
        arg_count = 0
        for i,c in enumerate(right_str):
            if c  == '{':
                if num == 0:
                    istart = i+1
                num += 1
            if c == '}':
                num -= 1
                if num == 0:
                    args.append(right_str[istart:i])
                    arg_count += 1
                    if arg_count == self.numargs:
                        right_str = right_str[i+1:]
                        break
        return args, left_str, right_str
    def replace_line(self, input_str):
        args, left_str, right_str = self.parse_arguments(input_str)
        return left_str + self.replace(args) + right_str

    def __repr__(self):
        return self.original + '\n' + self.name + '\t\t' + str(self.numargs) + '\t\t'+ self.definition
    
    
def process_string(input_str, macro_list):
    res = input_str
    while sum([res.count(m.name) for m in macro_list]) > 0:
        for m in macro_list:
            if m.name in res:
                res = m.replace_line(res)
    return res
    
def process_file(filename,macro_list):
    with open(filename,'r') as f:
        with open(filename+'.temp','w') as g:
            for line in f.readlines():
                while sum([line.count(m.name) for m in macro_list]) > 0:
                    for m in macro_list:
                        if m.name in line:
                            line = m.replace_line(line)
                g.write(line)

def process_project(filename):
    with open(filename,'r') as f:
        with open(filename + '.temp','w') as g:
            lines = f.readlines()
            newlines = []
            project_files =[filename]
            for line in lines:
                if 'newcommand' in line:
                    macro_list.append(line)
                else:
                    newlines.append(line)    
def grab_argument(input_str, numargs, bracket_type ='{}'):
        left_bracket, right_bracket = bracket_type
        temp = input_str
        
        num = 0
        istart = 0
        args = []
        arg_count = 0
        for i,c in enumerate(temp):
            if c  == left_bracket:
                if num == 0:
                    istart = i+1
                num += 1
            if c == right_bracket:
                num -= 1
                if num == 0:
                    args.append(temp[istart:i])
                    arg_count += 1
                    if arg_count == numargs:
                        break
        return args

In [3]:
%%bash
cd tests/
build.sh

Latexmk: applying rule 'pdflatex'...
This is pdfTeX, Version 3.14159265-2.6-1.40.16 (TeX Live 2015) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./main.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(./emulateapj.cls
Document Class: emulateapj 
(/usr/local/texlive/2015/texmf-dist/tex/latex/revtex/revtex4-1.cls
Document Class: revtex4-1 2010/07/25/20:33:00 4.1r (http://publish.aps.org/revt
ex4/ for documentation)
ltxutil[2010/07/25/20:33:00 4.1r utilities package (portions licensed from W. E
. Baxter web at superscript.com)]
ltxfront[2010/07/25/20:33:00 4.1r frontmatter package (AO,DPC)]
ltxgrid[2010/07/25/20:33:00 4.1r page grid package (portions licensed from W. E
. Baxter web at superscript.com)]
(/usr/local/texlive/2015/texmf-dist/tex/latex/revtex/aps4-1.rtx)
(/usr/local/texlive/2015/texmf-dist/tex/latex/revtex/aps10pt4-1.rtx)
(/usr/local/texlive/2015/texmf-dist/tex/latex/textcase/textcase.sty)
(/usr/local

Latexmk: This is Latexmk, John Collins, 5 February 2015, version: 4.43a.
Latexmk: Changing directory to './'
Rule 'pdflatex': Rules & subrules not known to be previously run:
   pdflatex
Rule 'pdflatex': The following rules & subrules became out-of-date:
      'pdflatex'
------------
Run number 1 of rule 'pdflatex'
------------
------------
Running 'pdflatex  -interaction=nonstopmode -synctex=1 -recorder  "main.tex"'
------------
Latexmk: Non-existent bbl file 'main.bbl'
 No file main.bbl.
Latexmk: References changed.
Latexmk: References changed.
Latexmk: References changed.
Latexmk: Log file says output to 'main.pdf'
Latexmk: List of undefined refs and citations:
  Citation `1973A&A....24..337S' on page 2 undefined on input line 57
  Citation `1974MNRAS.168..603L' on page 1 undefined on input line 31
  Citation `1979ApJ...233..857G' on page 1 undefined on input line 35
  Citation `1980ApJ...241..425G' on page 1 undefined on input line 35
  Citation `1986ApJ...309..846L' on page 1 unde

In [85]:
macs = [Macro(m) for m in macro_list]
names = [m.name for m in macs]
for m in macs:
    print(m)
#print(macro_list)
#[print(Macro(m).name) for m in macro_list]

#print(mdat.replace(('S1','S2')))

\renewcommand*{\sectionautorefname}{Section}
\sectionautorefname		0		Section
\newcommand{\pderiv}[1]{\frac{\partial}{\partial #1}}
\pderiv		1		\frac{\partial}{\partial #1}
\newcommand{\ppderiv}[2]{\frac{\partial #1}{\partial #2}}
\ppderiv		2		\frac{\partial #1}{\partial #2}
\newcommand{\deriv}[1]{\frac{d}{d #1}}
\deriv		1		\frac{d}{d #1}
\newcommand{\cderiv}[1]{\frac{D}{D #1}}
\cderiv		1		\frac{D}{D #1}
\newcommand{\del}{\mathbf{\nabla}}
\del		0		\mathbf{\nabla}
\renewcommand{\div}{\del \cdot}
\div		0		\del \cdot
\newcommand{\avg}[1]{\left\langle #1 \right\rangle}
\avg		1		\left\langle #1 \right\rangle
\renewcommand{\vec}[1]{\mathbf{#1}}
\vec		1		\mathbf{#1}
\newcommand{\delr}[1]{\frac{1}{r} \pderiv{r} \left( r #1 \right)}
\delr		1		\frac{1}{r} \pderiv{r} \left( r #1 \right)
\newcommand{\delp}[1]{\frac{1}{r} \pderiv{\phi} \left( #1 \right)}
\delp		1		\frac{1}{r} \pderiv{\phi} \left( #1 \right)
\newcommand{\fargo}{FARGO3D }
\fargo		0		FARGO3D 


In [15]:
for m in macs:
    print(m.name+ '\t' + str(m.numargs) + '\t' + m.definition)

\sectionautorefname	0	Section
\pderiv	1	\frac{\partial}{\partial #1}
\ppderiv	2	\frac{\partial #1}{\partial #2}
\deriv	1	\frac{d}{d #1}
\cderiv	1	\frac{D}{D #1}
\del	0	\mathbf{\nabla}
\div	0	\del \cdot
\avg	1	\left\langle #1 \right\rangle
\vec	1	\mathbf{#1}
\delr	1	\frac{1}{r} \pderiv{r} \left( r #1 \right)
\delp	1	\frac{1}{r} \pderiv{\phi} \left( #1 \right)
\fargo	0	FARGO3D 


In [128]:
s1,s2 = ['1','2']

In [168]:
name, numargs, dat

('\\ppderiv', 2, '\\frac{\\partial {0}}{\\partial {1}}')

In [286]:
macro_list

['\\renewcommand*{\\sectionautorefname}{Section}\n',
 '\\newcommand{\\pderiv}[1]{\\frac{\\partial}{\\partial #1}}\n',
 '\\newcommand{\\ppderiv}[2]{\\frac{\\partial #1}{\\partial #2}}\n',
 '\\newcommand{\\deriv}[1]{\\frac{d}{d #1}}\n',
 '\\newcommand{\\cderiv}[1]{\\frac{D}{D #1}}\n',
 '\\newcommand{\\del}{\\mathbf{\\nabla}}\n',
 '\\renewcommand{\\div}{\\del \\cdot}\n',
 '\\newcommand{\\avg}[1]{\\left\\langle #1 \\right\\rangle}\n',
 '\\renewcommand{\\vec}[1]{\\mathbf{#1}}\n',
 '\\newcommand{\\delr}[1]{\\frac{1}{r} \\pderiv{r} \\left( r #1 \\right)}\n',
 '\\newcommand{\\delp}[1]{\\frac{1}{r} \\pderiv{\\phi} \\left( #1 \\right)}\n',
 '\\newcommand{\\fargo}{FARGO3D }\n']

In [16]:
line_list = []
for line in lines:
    for m in macs:
        if m.name in line:
            line_list.append(line)
            
            

In [55]:
args, res, temp = macs[2].parse_arguments(line_list[4])
res2 = macs[2].replace(args)

In [30]:
temp = line_list[4].strip().split(macs[2].name)[1]

In [57]:
print(args)
print(''.join([res2,temp]))

['\\Sigma', 't']
\frac{\partial \Sigma}{\partial t} + \div \left( \Sigma \vec{v} \right) .


In [62]:
print(line_list[4])
print(macs[2].replace_line(line_list[4]))

\ppderiv{\Sigma}{t} + \div \left( \Sigma \vec{v} \right) .

\frac{\partial \Sigma}{\partial t} + \div \left( \Sigma \vec{v} \right) .


In [69]:
res = []
for line in line_list:
    temp = line
    while sum([temp.count(m.name) for m in macs]) > 0:
        for m in macs:
            if m.name in temp:
                temp = m.replace_line(temp)
    res.append(temp)


In [32]:
with open('results.dat','w') as f:
    for i in range(len(line_list)):
        f.write('################\n' + line_list[0] + '\n-----------------\n' + res[0] + '\n################\n')

In [70]:
print(line_list[0])
print(res[0])

\pderiv{t} \left( \Sigma r^2 \Omega \right) + \div \left( \Sigma  v_r r^2 \Omega - \vec{\Pi_{r\phi}} \right) = - \Sigma r \del \Phi ,

\frac{\partial}{\partial t} \left( \Sigma r^2 \Omega \right) + \mathbf{\nabla} \cdot \left( \Sigma  v_r r^2 \Omega - \mathbf{\Pi_{r\phi}} \right) = - \Sigma r \mathbf{\nabla} \Phi ,


In [95]:
with open('tests/origs/main.tex','r') as f:
    lines = f.read()

In [96]:
lines.split('renewcommand')

['% !TEX TS-program = pdflatexmk\n\\documentclass[iop,onecolumn,numberedappendix,appendixfloats]{emulateapj}\n\\usepackage{amsmath,amssymb,amstext}\n\n\\usepackage[breaklinks,colorlinks,citecolor=blue,linkcolor=magenta]{hyperref} \n\\',
 '*{\\sectionautorefname}{Section}\n\\usepackage[all]{hypcap} %Links go ts; breaks on deluxetables (use \\capstartfalse \\capstarttrue to fix it)\n\n\\newcommand{\\pderiv}[1]{\\frac{\\partial}{\\partial #1}}\n\\newcommand{\\ppderiv}[2]{\\frac{\\partial #1}{\\partial #2}}\n\\newcommand{\\deriv}[1]{\\frac{d}{d #1}}\n\\newcommand{\\cderiv}[1]{\\frac{D}{D #1}}\n\\newcommand{\\del}{\\mathbf{\\nabla}}\n\\',
 '{\\div}{\\del \\cdot}\n\\newcommand{\\avg}[1]{\\left\\langle #1 \\right\\rangle}\n\\',
 '{\\vec}[1]{\\mathbf{#1}}\n\\newcommand{\\delr}[1]{\\frac{1}{r} \\pderiv{r} \\left( r #1 \\right)}\n\\newcommand{\\delp}[1]{\\frac{1}{r} \\pderiv{\\phi} \\left( #1 \\right)}\n\\newcommand{\\fargo}{FARGO3D }\n\n\\usepackage{aas_macros}\n\\usepackage{natbib}\n\\bibliogr

In [92]:
res = process_string(lines,macs)

In [93]:
print(lines)

\section{The deposited torque}

The equations of motion for the total angular momentum density and surface density are,

\begin{equation} \label{eq:tot_ang}
\pderiv{t} \left( \Sigma r^2 \Omega \right) + \div \left( \Sigma  v_r r^2 \Omega - \vec{\Pi_{r\phi}} \right) = - \Sigma r \del \Phi ,
\end{equation}
\begin{equation} \label{eq:tot_sigma}
\ppderiv{\Sigma}{t} + \div \left( \Sigma \vec{v} \right) .
\end{equation}

Our goal is to determine the evolution of the azimuthally-averaged surface density, $\avg{\Sigma}$. In particular, we are interested in the \emph{steady-state} surface density as a function of the disk's viscosity, mass flux, and the mass of the embedded planet. The evolution of the averaged surface density follows,
\begin{equation} \label{eq:avg_sigma}
\pderiv{t}\avg{2 \pi r \Sigma} + \pderiv{r} \avg{2 \pi r \Sigma v_r}  = 0 .
\end{equation}

We will obtain an expression for the radial mass flux, $\dot{M} = \avg{-2 \pi r \Sigma v_r}$, that depends on the rate at which angul

In [94]:
print(res)

\section{The deposited torque}

The equations of motion for the total angular momentum density and surface density are,

\begin{equation} \label{eq:tot_ang}
\frac{\partial}{\partial t} \left( \Sigma r^2 \Omega \right) + \mathbf{\nabla} \cdot \left( \Sigma  v_r r^2 \Omega - \mathbf{\Pi_{r\phi}} \right) = - \Sigma r \mathbf{\nabla} \Phi ,
\end{equation}
\begin{equation} \label{eq:tot_sigma}
\frac{\partial \Sigma}{\partial t} + \mathbf{\nabla} \cdot \left( \Sigma \mathbf{v} \right) .
\end{equation}

Our goal is to determine the evolution of the azimuthally-averaged surface density, $\left\langle \Sigma \right\rangle$. In particular, we are interested in the \emph{steady-state} surface density as a function of the disk's viscosity, mass flux, and the mass of the embedded planet. The evolution of the averaged surface density follows,
\begin{equation} \label{eq:avg_sigma}
\frac{\partial}{\partial t}\left\langle 2 \pi r \Sigma \right\rangle + \frac{\partial}{\partial r} \left\langle 2 \pi r \

In [152]:
print(lines)

% !TEX TS-program = pdflatexmk
\documentclass[iop,onecolumn,numberedappendix,appendixfloats]{emulateapj}
\usepackage{amsmath,amssymb,amstext}

\usepackage[breaklinks,colorlinks,citecolor=blue,linkcolor=magenta]{hyperref} 
\renewcommand*{\sectionautorefname}{Section}
\usepackage[all]{hypcap} %Links go ts; breaks on deluxetables (use \capstartfalse \capstarttrue to fix it)

\newcommand{\pderiv}[1]{\frac{\partial}{\partial #1}}
\newcommand{\ppderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\deriv}[1]{\frac{d}{d #1}}
\newcommand{\cderiv}[1]{\frac{D}{D #1}}
\newcommand{\del}{\mathbf{\nabla}}
\renewcommand{\div}{\del \cdot}
\newcommand{\avg}[1]{\left\langle #1 \right\rangle}
\renewcommand{\vec}[1]{\mathbf{#1}}
\newcommand{\delr}[1]{\frac{1}{r} \pderiv{r} \left( r #1 \right)}
\newcommand{\delp}[1]{\frac{1}{r} \pderiv{\phi} \left( #1 \right)}
\newcommand{\fargo}{FARGO3D }

\usepackage{aas_macros}
\usepackage{natbib}
\bibliographystyle{apj}

\shorttitle{Short Title}
\shortauthors{Dempsey

In [150]:
mlist= [Macro('newcommand'+temp) for temp in lines.split('\\newcommand')[1:-1]]


In [156]:
Macro('newcommand'+lines.split('\\newcommand')[-1])

newcommand{\fargo}{FARGO3D }

\usepackage{aas_macros}
\usepackage{natbib}
\bibliographystyle{apj}

\shorttitle{Short Title}
\shortauthors{Dempsey \& Lithwick}

\begin{document}

\title{Criteria for Inner Cavity Opening}
\author{Adam M. Dempsey \& Yoram Lithwick}
\affil{Affiliations}

\begin{abstract}
Abstract.
\end{abstract}

\keywords{keywords}
\maketitle



\input{theory}

\input{methods}



\acknowledgments{
}
\bibliography{migrationbib}


\begin{appendix}

\end{appendix}





\end{document}
\fargo		0		FARGO3D }

\usepackage{aas_macros}
\usepackage{natbib}
\bibliographystyle{apj}

\shorttitle{Short Title}
\shortauthors{Dempsey \& Lithwick}

\begin{document}

\title{Criteria for Inner Cavity Opening}
\author{Adam M. Dempsey \& Yoram Lithwick}
\affil{Affiliations}

\begin{abstract}
Abstract.
\end{abstract}

\keywords{keywords}
\maketitle



\input{theory}

\input{methods}



\acknowledgments{
}
\bibliography{migrationbib}


\begin{appendix}

\end{appendix}





\end{document

In [136]:
fnames = [grab_argument(line,1)[0] + '.tex' for line in lines.split('input')[1:] ]

In [151]:
for m in mlist:
    print(m.name + '\t\t' + m.definition + '\t\t{:d}'.format(m.numargs))

\pderiv		\frac{\partial}{\partial #1}		1
\ppderiv		\frac{\partial #1}{\partial #2}		2
\deriv		\frac{d}{d #1}		1
\cderiv		\frac{D}{D #1}		1
\del		\mathbf{\nabla}		0
\avg		\left\langle #1 \right\rangle		1
\delr		\frac{1}{r} \pderiv{r} \left( r #1 \right)		1
\delp		\frac{1}{r} \pderiv{\phi} \left( #1 \right)		1
